# Anexo V

## Librerias necesarias

In [1]:
import pandas as pd
import os
import pickle
import xlsxwriter
from xlsxwriter.utility import xl_range, xl_rowcol_to_cell
import re
from typing import Dict, List, Any
import itertools
from firebase_admin import credentials, firestore
import excel2img
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import os
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from google.oauth2 import service_account
from google.cloud import firestore
import json
from datetime import date, timedelta
import calendar
import math
import random
from typing import List, Dict, Optional

## Datos del proyecto en análisis

In [2]:
proyecto="cabanaconde"
mes=4
anio=2025

## Datos de Firestore

In [3]:
## Datos de # 1. Busca el .env en el directorio actual o en cualquiera de los padres
dotenv_path = find_dotenv()
if not dotenv_path:
    raise FileNotFoundError("No se encontró ningún archivo .env en este directorio ni en sus padres.")
load_dotenv(dotenv_path)

# 2. Define el root del proyecto como la carpeta que contiene el .env
project_root = Path(dotenv_path).parent

# 3. Obtén la ruta relativa de las credenciales desde la variable de entorno
rel_cred_path = os.getenv("FIRESTORE_CREDENTIALS")
if not rel_cred_path:
    raise RuntimeError("No existe la variable FIRESTORE_CREDENTIALS en el .env")

# 4. Construye la ruta absoluta al JSON
cred_path = Path(rel_cred_path)
if not cred_path.is_absolute():
    cred_path = (project_root / cred_path).resolve()

if not cred_path.exists():
    raise FileNotFoundError(f"No existe el archivo de credenciales en: {cred_path}")

# 5. Carga las credenciales y crea el cliente de Firestore
credentials = service_account.Credentials.from_service_account_file(str(cred_path))
client = firestore.Client(credentials=credentials, project=credentials.project_id)

# 6. Prueba que funcione
print("Colecciones disponibles:", [c.id for c in client.collections()])

Colecciones disponibles: ['rutinarios']


In [4]:
db=firestore.Client(credentials=credentials, project=credentials.project_id)

In [5]:
# 4. Define la ruta a tu documento anidado
colec_raiz = "rutinarios"
doc_proyecto = proyecto     # puede ser tu variable proyecto
# colec_valoriz = "valorizaciones"
# id_valoriz = "4"  

doc_ref = (
    db
    .collection(colec_raiz)
    .document(doc_proyecto)
    # .collection(colec_valoriz)
    # .document(id_valoriz)
)

data_mantenimiento_snapshot=doc_ref.get()

if not data_mantenimiento_snapshot.exists:
    print(f"El documento {doc_ref.path} no existe.")

data_mantenimiento= data_mantenimiento_snapshot.to_dict()
print(data_mantenimiento)

{'otros': {'tipo_superficie': 'afirmado', 'estado_conservacion_via': 'bueno'}, 'datos_generales': {'distritos': ['Cabanaconde'], 'region': 'Arequipa', 'provincia': 'Caylloma'}, 'expediente': {'codigo_ruta': 'R0405106', 'monto_total': 81120, 'coordenadas': {'fin': {'hemisferio': 'S', 'y': 8272525.68, 'progresiva': 13000, 'zona_letra': 'L', 'x': 813907.99, 'datum': 'WGS84', 'altitud': 1975, 'zona': 18}, 'inicio': {'hemisferio': 'S', 'y': 8272658.46, 'progresiva': 0, 'zona_letra': 'L', 'x': 819845.05, 'datum': 'WGS84', 'altitud': 2877, 'zona': 18}}, 'nombre': 'MANTENIMIENTO VIAL RUTINARIO DEL CAMINO VECINAL EMP. R0405104 (CABANACONDE) A PTE. CABANACONDE - CHOCO (L.P. CASTILLA), DISTRITO DE CABANACONDE, PROVINCIA DE CAYLLOMA, REGIÓN AREQUIPA', 'tarifa_km_mes': 780, 'longitud': 13000}, 'other': {'tipo_superficie': 'afirmado', 'estado_conservacion_via': 'bueno'}, 'contrato': {'jefe_mantenimiento': {'titulo': 'Ingeniero', 'nombre': 'Genaro', 'dni': 0, 'apellido': 'Tinta Cáceres'}, 'numero_tra

In [6]:
longitud=data_mantenimiento['expediente']['longitud']
print(f"Longitud del contrato: {longitud} m")

Longitud del contrato: 13000 m


In [7]:
empresa=data_mantenimiento['contrato']['contratista']['razon_social']
print(f"Empresa contratista: {empresa}")

Empresa contratista: Grupo ARICOL E.I.R.L.


In [8]:
tramo=data_mantenimiento['contrato']['denominacion_tramo_convenio']
print(f"Tramo: {tramo}")

Tramo: Emp. Choco Soro Soro (Cabanaconde) a Pte. Cabanaconde - Choco - Ld Castilla (Km 14+064)


In [9]:
sector=''.join(data_mantenimiento['datos_generales']['distritos'])
print(f"Sector: {sector}")

Sector: Cabanaconde


## Cargando actividades

In [10]:
ruta_actividades= os.path.join("data", "general_data", "actividades.json")
with open(ruta_actividades, 'r', encoding='utf-8') as archivo:
    actividades = json.load(archivo)
# Ahora 'datos' es un diccionario de Python
print(actividades)

[{'key': 'MR100', 'value': {'label': 'Conservación de calzada', 'value': [{'key': 'MR101', 'value': {'label': 'Limpieza de calzada', 'carga_trabajo': 0, 'unidad': 'Km'}}, {'key': 'MR102', 'value': {'label': 'Bacheo', 'carga_trabajo': 0, 'unidad': 'm2'}}, {'key': 'MR103', 'value': {'label': 'Desquinche', 'carga_trabajo': 0, 'unidad': 'm3'}}, {'key': 'MR104', 'value': {'label': 'Remoción de derrumbes', 'carga_trabajo': 0, 'unidad': 'm3'}}]}}, {'key': 'MR200', 'value': {'label': 'Limpieza de obras de arte', 'value': [{'key': 'MR201', 'value': {'label': 'Limpieza de cunetas', 'carga_trabajo': 1440, 'unidad': 'm'}}, {'key': 'MR202', 'value': {'label': 'Limpieza de alcantarillas', 'carga_trabajo': 0, 'unidad': 'unidad'}}, {'key': 'MR203', 'value': {'label': 'Limpieza de badén', 'carga_trabajo': 0, 'unidad': 'm2'}}, {'key': 'MR204', 'value': {'label': 'Limpieza de zanjas de coronación', 'carga_trabajo': 0, 'unidad': 'm'}}, {'key': 'MR205', 'value': {'label': 'Limpieza de pontones', 'carga_tra

## Funciones útiles

In [11]:
def distribuir_plazos(fecha_inicio: date, duracion: int) -> list[dict]:
    """
    Divide el periodo de obra en tramos mensuales.

    Parámetros:
    - fecha_inicio: date en que inicia la obra (se cuenta ese día como parte del plazo).
    - duracion: número de días calendarios (> 0).

    Devuelve:
    Una lista de diccionarios, uno por cada mes cubierto, con la forma:
      {
        "desde": {"year": int, "month": int, "day": int, "date": date},
        "hasta": {"year": int, "month": int, "day": int, "date": date}
      }

    Lanza ValueError si:
    - duracion ≤ 0
    - la fecha de fin (incluyendo el día de arranque) cruza al año siguiente.
    """
    if duracion <= 0:
        raise ValueError(f"Duración inválida ({duracion}); debe ser un entero positivo.")

    # Cálculo de la fecha final (inclusiva)
    fecha_fin = fecha_inicio + timedelta(days=duracion - 1)
    if fecha_fin.year > fecha_inicio.year:
        raise ValueError(
            f"La obra se extiende hasta {fecha_fin}, excede el año {fecha_inicio.year}."
        )

    resultados = []
    # Iterar mes a mes desde fecha_inicio hasta fecha_fin
    año, mes = fecha_inicio.year, fecha_inicio.month
    while True:
        # Primer día del mes actual
        inicio_mes = date(año, mes, 1)
        # Último día del mes actual
        ultimo_dia = calendar.monthrange(año, mes)[1]
        fin_mes = date(año, mes, ultimo_dia)

        # Determinar tramo "desde" y "hasta" dentro de este mes
        tramo_desde = fecha_inicio if (año == fecha_inicio.year and mes == fecha_inicio.month) else inicio_mes
        tramo_hasta = fecha_fin if (año == fecha_fin.year and mes == fecha_fin.month) else fin_mes

        resultados.append({
            "desde": {
                "year": tramo_desde.year,
                "month": tramo_desde.month,
                "day": tramo_desde.day,
                "date": tramo_desde
            },
            "hasta": {
                "year": tramo_hasta.year,
                "month": tramo_hasta.month,
                "day": tramo_hasta.day,
                "date": tramo_hasta
            }
        })

        # Si ya cubrimos hasta la fecha_fin, rompemos
        if (año, mes) == (fecha_fin.year, fecha_fin.month):
            break

        # Avanzar al siguiente mes
        if mes == 12:
            año += 1
            mes = 1
        else:
            mes += 1

    return resultados

### Generador de errores

In [12]:
def generar_datos_errores_unidades(
    longitud: float,
    error_permitido: float,
    recurrencia: float,
    intensidad: float,
    variacion: float,
    paso: float,
    seed: Optional[int] = None
) -> List[Dict]:
    """
    Genera datos falsos de errores distribuidos en tramos de carretera.

    Parámetros:
    - longitud (float): Longitud total de la carretera en metros. Debe ser > 0.
    - error_permitido (float): Número máximo de fallas por kilómetro (contador ≥ 0).
    - recurrencia (float): Proporción de tramos que tendrán error (0 ≤ recurrencia ≤ 1).
    - intensidad (float): Factor que multiplica el máximo de errores permitidos (≥ 0).
    - variacion (float): Amplitud de variación relativa sobre el valor base (0 ≤ variacion ≤ 1).
    - paso (float): Incremento mínimo para el valor de error (> 0).
    - seed (int, opcional): Semilla para reproducibilidad. Por defecto None.

    Validaciones:
    - longitud > 0
    - error_permitido ≥ 0
    - 0 ≤ recurrencia ≤ 1
    - intensidad ≥ 0
    - 0 ≤ variacion ≤ 1
    - paso > 0

    Salida:
    Lista de diccionarios, uno por tramo. Cada dict contiene:
    - "tramo": número de tramo (1, 2, …)
    - "inicio_m": punto de inicio del tramo (en metros)
    - "fin_m": punto final del tramo (en metros)
    - "longitud_m": longitud efectiva del tramo (m)
    - "error": valor simulado de errores (múltiplo de `paso`)
    """
    # Validaciones de entrada
    if longitud <= 0:
        raise ValueError("La longitud debe ser > 0.")
    if error_permitido < 0:
        raise ValueError("error_permitido debe ser ≥ 0.")
    if not (0 <= recurrencia <= 1):
        raise ValueError("recurrencia debe estar entre 0 y 1.")
    if intensidad < 0:
        raise ValueError("intensidad debe ser ≥ 0.")
    if not (0 <= variacion <= 1):
        raise ValueError("variacion debe estar entre 0 y 1.")
    if paso <= 0:
        raise ValueError("paso debe ser > 0.")

    # Fijar semilla si se pide
    if seed is not None:
        random.seed(seed)

    # División en tramos de 1 km
    num_tramos = math.ceil(longitud / 1000)
    tramos = []
    for i in range(num_tramos):
        inicio = i * 1000.0
        fin = min((i + 1) * 1000.0, longitud)
        tramos.append({
            "tramo": i + 1,
            "inicio_m": inicio,
            "fin_m": fin,
            "longitud_m": fin - inicio,
            "error": 0.0  # se llenará luego
        })

    # Número de tramos con error
    errores_tramos = round(num_tramos * recurrencia)

    # Selección "espaciada" de índices
    indices_error = set()
    if errores_tramos > 0:
        paso_espacio = num_tramos / errores_tramos
        for k in range(errores_tramos):
            idx = int(math.floor((k + 0.5) * paso_espacio))
            # asegurar rango válido
            idx = max(0, min(num_tramos - 1, idx))
            indices_error.add(idx)

    # Generación de valores de error
    base_error = intensidad * error_permitido
    for idx in indices_error:
        # variación aleatoria en ±variacion * base_error
        delta = random.uniform(-variacion, variacion) * base_error
        raw = base_error + delta
        # redondear al múltiplo de `paso`
        ajustado = round(raw / paso) * paso
        # evitar negativos
        tramos[idx]["error"] = max(0.0, ajustado)

    return tramos

In [ ]:
import math
import random
from typing import List, Dict, Optional

def generar_datos_falla_longitud(
    longitud: float,
    max_longitud_falla_km: float,
    recurrencia: float,
    intensidad: float,
    variacion: float,
    paso: float,
    seed: Optional[int] = None
) -> List[Dict]:
    """
    Genera datos falsos de longitud de falla distribuidos en tramos de carretera.

    Parámetros:
    - longitud (float): longitud total de la carretera en metros (> 0).
    - max_longitud_falla_km (float): longitud máxima de falla en metros por kilómetro (≥ 0).
    - recurrencia (float): proporción de tramos que tendrán falla (0 ≤ recurrencia ≤ 1).
    - intensidad (float): factor multiplicador sobre la longitud máxima (≥ 0).
    - variacion (float): ±porcentaje de variación sobre el valor base (0 ≤ variacion ≤ 1).
    - paso (float): resolución mínima en metros; el fallo se redondeará a múltiplos de este paso (> 0).
    - seed (int, opcional): semilla para reproducibilidad.

    Devuelve:
    Lista de dicts, uno por tramo, con estas claves:
      - "tramo": número de tramo (1, 2, …)
      - "inicio_m", "fin_m", "longitud_m": en metros
      - "falla_m": longitud simulada de falla en ese tramo (múltiplo de `paso`)
    """
    # 1. Validaciones
    if longitud <= 0:
        raise ValueError("La longitud debe ser > 0.")
    if max_longitud_falla_km < 0:
        raise ValueError("max_longitud_falla_km debe ser ≥ 0.")
    if not (0 <= recurrencia <= 1):
        raise ValueError("recurrencia debe estar entre 0 y 1.")
    if intensidad < 0:
        raise ValueError("intensidad debe ser ≥ 0.")
    if not (0 <= variacion <= 1):
        raise ValueError("variacion debe estar entre 0 y 1.")
    if paso <= 0:
        raise ValueError("paso debe ser > 0.")

    # 2. Semilla
    if seed is not None:
        random.seed(seed)

    # 3. División en tramos de 1 km
    num_tramos = math.ceil(longitud / 1000)
    tramos = []
    for i in range(num_tramos):
        inicio = i * 1000.0
        fin = min((i + 1) * 1000.0, longitud)
        tramos.append({
            "tramo": i + 1,
            "inicio_m": inicio,
            "fin_m": fin,
            "longitud_m": fin - inicio,
            "falla_m": 0.0
        })

    # 4. Cuántos tramos deben fallar
    n_fallas = round(num_tramos * recurrencia)

    # 5. Selección “espaciada” de índices con falla
    indices = set()
    if n_fallas > 0:
        espacio = num_tramos / n_fallas
        for k in range(n_fallas):
            idx = int(math.floor((k + 0.5) * espacio))
            idx = max(0, min(num_tramos - 1, idx))
            indices.add(idx)

    # 6. Generación de la longitud de falla
    base = intensidad * max_longitud_falla_km
    for idx in indices:
        delta = random.uniform(-variacion, variacion) * base
        raw = base + delta
        ajustado = round(raw / paso) * paso
        tramos[idx]["falla_m"] = max(0.0, ajustado)

    return tramos


In [15]:
import math
import random
from typing import List, Dict, Optional

def generar_datos_porcentaje(
    longitud: float,
    max_porcentaje_km: float,
    recurrencia: float,
    intensidad: float,
    variacion: float,
    paso: float,
    seed: Optional[int] = None
) -> List[Dict]:
    """
    Genera datos falsos de porcentaje de sección bloqueada distribuidos en tramos de carretera.

    Parámetros:
    - longitud (float): longitud total en metros (> 0).
    - max_porcentaje_km (float): porcentaje máximo bloqueado por km (0 ≤ valor ≤ 1).
    - recurrencia (float): proporción de tramos con bloqueo (0 ≤ valor ≤ 1).
    - intensidad (float): factor multiplicador sobre el porcentaje máximo (≥ 0).
    - variacion (float): ±porcentaje de variación sobre el valor base (0 ≤ valor ≤ 1).
    - paso (float): resolución mínima en proporción (p. ej. 0.01 para 1%) (> 0).
    - seed (int, opcional): semilla para reproducibilidad.

    Devuelve:
    Lista de dicts (uno por tramo) con estas claves:
      - "tramo": número de tramo (1, 2, …)
      - "inicio_m", "fin_m", "longitud_m": metros
      - "bloqueado_pct": proporción bloqueada en [0, 1], múltiplo de `paso`
    """
    # Validaciones
    if longitud <= 0:
        raise ValueError("La longitud debe ser > 0.")
    if not (0 <= max_porcentaje_km <= 1):
        raise ValueError("max_porcentaje_km debe estar entre 0 y 1.")
    if not (0 <= recurrencia <= 1):
        raise ValueError("recurrencia debe estar entre 0 y 1.")
    if intensidad < 0:
        raise ValueError("intensidad debe ser ≥ 0.")
    if not (0 <= variacion <= 1):
        raise ValueError("variacion debe estar entre 0 y 1.")
    if paso <= 0 or paso > 1:
        raise ValueError("paso debe ser > 0 y ≤ 1.")

    if seed is not None:
        random.seed(seed)

    # División en tramos de 1 km
    num_tramos = math.ceil(longitud / 1000)
    tramos = []
    for i in range(num_tramos):
        inicio = i * 1000.0
        fin = min((i + 1) * 1000.0, longitud)
        tramos.append({
            "tramo": i + 1,
            "inicio_m": inicio,
            "fin_m": fin,
            "longitud_m": fin - inicio,
            "bloqueado_pct": 0.0
        })

    # Cuántos tramos tendrán bloqueo
    n_bloqueos = round(num_tramos * recurrencia)

    # Selección “espaciada”
    indices = set()
    if n_bloqueos > 0:
        espacio = num_tramos / n_bloqueos
        for k in range(n_bloqueos):
            idx = int(math.floor((k + 0.5) * espacio))
            idx = max(0, min(num_tramos - 1, idx))
            indices.add(idx)

    # Generación de porcentaje bloqueado
    base = intensidad * max_porcentaje_km
    for idx in indices:
        delta = random.uniform(-variacion, variacion) * base
        raw = base + delta
        ajustado = round(raw / paso) * paso
        # Clamp a [0, 1]
        tramos[idx]["bloqueado_pct"] = min(1.0, max(0.0, ajustado))

    return tramos


In [94]:
actividades={
    '101':{
        "error_permitido":3,
        "recurrencia":0.15,
        "intensidad":0.4,
        "variacion":0.5,
        "paso":0.25,
    },
    '102':{
        "error_permitido":10,
        "recurrencia":0.25,
        "intensidad":0.25,
        "variacion":0.25,
        "paso":0.5,
    },
    '104':{
        "error_permitido":1,
        "recurrencia":0.5,
        "intensidad":0.2,
        "variacion":0.6,
        "paso":0.1,
    },
    '201':{
        "error_permitido":25,
        "recurrencia":0.6,
        "intensidad":0.1,
        "variacion":0.6,
        "paso":1,
    },
    '203':{
        "error_permitido":30,
        "recurrencia":0.4,
        "intensidad":0.25,
        "variacion":0.5,
        "paso":1,
    },
    '301':{
        "error_permitido":45,
        "recurrencia":0.8,
        "intensidad":0.15,
        "variacion":0.8,
        "paso":1,
    },
    '401':{
        "error_permitido":1,
        "recurrencia":0.15,
        "intensidad":1,
        "variacion":0,
        "paso":0.1,
    },
    '701':{
        "error_permitido":5,
        "recurrencia":0.25,
        "intensidad":0.25,
        "variacion":0.4,
        "paso":0.25,
    },
}

In [129]:
actividad_en_curso = '701'  # Por ejemplo, la actividad 101

In [130]:
error= actividades[actividad_en_curso]["error_permitido"]
recurrencia = actividades[actividad_en_curso]["recurrencia"]
intensidad = actividades[actividad_en_curso]["intensidad"]
variacion = actividades[actividad_en_curso]["variacion"]
paso = actividades[actividad_en_curso]["paso"]

In [131]:
datos_unidades = generar_datos_errores_unidades(
    longitud=6600.0,
    error_permitido=error,
    recurrencia=recurrencia,
    intensidad=intensidad,
    variacion=variacion,
    paso=paso,
    seed=23
)

# seed=42 cabanaconde
# seed=23  callacalla
# for d in datos_unidades:
#     print(d)

# print(datos_unidades)

### Texto para pegar en excel

In [132]:
texto_para_excel = '\t'.join(str(round(item['error'], 2)) for item in datos_unidades)

# Mostrar sin print para facilitar copiar
from IPython.display import display, HTML
display(HTML(f"<textarea rows=3 cols=100>{texto_para_excel}</textarea>"))

## Cálculos

In [ ]:
inicio = date(2025, 4, 15)
dur = 60
for tramo in distribuir_plazos(inicio, dur):
    d = tramo["desde"]["date"]
    h = tramo["hasta"]["date"]
    print(f"{d} → {h}")

2025-04-15 → 2025-04-30
2025-05-01 → 2025-05-31
2025-06-01 → 2025-06-13


In [ ]:
datos_unidades = generar_datos_errores_unidades(
    longitud=longitud,
    error_permitido=3,
    recurrencia=0.1,
    intensidad=0.5,
    variacion=0.2,
    paso=0.25,
    seed=42
)
for d in datos_unidades:
    print(d)

{'tramo': 1, 'inicio_m': 0.0, 'fin_m': 1000.0, 'longitud_m': 1000.0, 'error': 0.0}
{'tramo': 2, 'inicio_m': 1000.0, 'fin_m': 2000.0, 'longitud_m': 1000.0, 'error': 0.0}
{'tramo': 3, 'inicio_m': 2000.0, 'fin_m': 3000.0, 'longitud_m': 1000.0, 'error': 0.0}
{'tramo': 4, 'inicio_m': 3000.0, 'fin_m': 4000.0, 'longitud_m': 1000.0, 'error': 0.0}
{'tramo': 5, 'inicio_m': 4000.0, 'fin_m': 5000.0, 'longitud_m': 1000.0, 'error': 0.0}
{'tramo': 6, 'inicio_m': 5000.0, 'fin_m': 6000.0, 'longitud_m': 1000.0, 'error': 0.0}
{'tramo': 7, 'inicio_m': 6000.0, 'fin_m': 7000.0, 'longitud_m': 1000.0, 'error': 1.5}
{'tramo': 8, 'inicio_m': 7000.0, 'fin_m': 8000.0, 'longitud_m': 1000.0, 'error': 0.0}
{'tramo': 9, 'inicio_m': 8000.0, 'fin_m': 9000.0, 'longitud_m': 1000.0, 'error': 0.0}
{'tramo': 10, 'inicio_m': 9000.0, 'fin_m': 10000.0, 'longitud_m': 1000.0, 'error': 0.0}
{'tramo': 11, 'inicio_m': 10000.0, 'fin_m': 11000.0, 'longitud_m': 1000.0, 'error': 0.0}
{'tramo': 12, 'inicio_m': 11000.0, 'fin_m': 12000.0,

## Escribiendo el excel

### Path

In [ ]:
# Ruta dinámica
ruta_directorio = os.path.join("output", proyecto, str(mes))
ruta_archivo = os.path.join(ruta_directorio, "anexo5.xlsx")
# Crear carpetas si no existen
os.makedirs(ruta_directorio, exist_ok=True)

wb = xlsxwriter.Workbook(ruta_archivo)
ws=wb.add_worksheet("anexo5")

### Formatos

In [ ]:
# 1. Propiedades base
BASE_CELL = {
    "valign": "vcenter",
    "border": 1,
}

BASE_HEADER = {
    **BASE_CELL,
    "bold": True,
}

BG_HEADER = "#D9E1F2"
BG_TABLE = "#C5D9F1"


# 2. Función de helper para crear formatos
def fmt(wb, **props):
    cfg = {}
    # parte genérica
    cfg.update(props)
    return wb.add_format(cfg)


# 3. Diccionario de formatos
formats = {
    # encabezados
    "header": {"align": "center", "bg_color": BG_HEADER, **BASE_HEADER},
    "header2": {"align": "left", **BASE_HEADER},
    "header_label": {"align": "center", **BASE_HEADER},
    # formato vertical header
    "header_vertical": {
        **BASE_HEADER,
        "bold": True,
        "align": "center",
        "valign": "vcenter",
        "bg_color": BG_HEADER,
        "rotation": 90,  # <— ¡aquí está el giro!
    },
    # encabezados de tabla
    "table_header": {
        "align": "center",
        "text_wrap": True,
        "shrink": True,
        "bg_color": BG_TABLE,
        **BASE_CELL,
    },
    "table_header_int": {"align": "center", "bg_color": BG_TABLE, **BASE_CELL},
    # celdas numéricas y de texto
    "cell": {"font_size": 8, "num_format": "#,##0.00", **BASE_CELL},
    # "cell": {"font_size": 8, "num_format": "#,##0.00", **BASE_CELL},
    "cell_int": {"num_format": "#,##0", **BASE_CELL},
    "cell_number": {"num_format": "#,##0.00", **BASE_CELL},
    "cell_number_percent": {
        "num_format": "#,##0.00%",
        "align": "right",
        **BASE_CELL,
    },
    "cell_text": {"align": "left", "text_wrap": True, "shrink": True, **BASE_CELL},
    "cell_right": {"align": "right", **BASE_CELL},
    # sumas
    "suma": {"num_format": "#,##0.00", **BASE_CELL},
    # soles
    "soles_color": {
        "bold": True,
        "bg_color": BG_HEADER,
        "num_format": '"S/." #,##0.00',
        **BASE_CELL,
    },
    "soles": {"num_format": '"S/." #,##0.00', **BASE_CELL},
    # formato de ajuste general
    "ajustar": {"align": "center", "text_wrap": True, "shrink": True, **BASE_CELL},
    "only_border": {
        "border": 1,
    },
}

# 4. Creación dinámica de los objetos Format
wb_formats = {name: fmt(wb, **props) for name, props in formats.items()}

# Ahora sólo usa wb_formats["header"], wb_formats["cell_text"], etc.

### Punto de inserción

In [ ]:
# Punto de inserción (cero-indexed)
table_start_row = 6   # si quieres que empiece en la fila 7
table_start_col = 1   # si quieres que empiece en la columna B

### Funciones de escritura de excel

In [ ]:
def merge(ws, r1, c1, r2, c2, valor, fmt):
    """Merge relative a table_start_* + offsets."""
    ws.merge_range(
        xl_range(table_start_row + r1,
                 table_start_col + c1,
                 table_start_row + r2,
                 table_start_col + c2),
        valor, fmt
    )

In [ ]:
def write_rel(ws, row_offset, col_offset, value, fmt=None):
    """
    Escribe en la celda (table_start_row + row_offset, table_start_col + col_offset)
    """
    abs_row = table_start_row + row_offset
    abs_col = table_start_col + col_offset
    ws.write(abs_row, abs_col, value, fmt)

In [ ]:
# 2. Helpers con tuplas (x, y)

def merge_xy(ws, start: tuple[int,int], end: tuple[int,int], value, fmt):
    """
    Merge de celdas desde start=(x1, y1) hasta end=(x2, y2),
    relativas a table_start_col/row.
    """
    (x1, y1), (x2, y2) = start, end
    ws.merge_range(
        xl_range(
            table_start_row + y1,  # row1 absoluto
            table_start_col + x1,  # col1 absoluto
            table_start_row + y2,  # row2 absoluto
            table_start_col + x2,  # col2 absoluto
        ),
        value, fmt
    )

def write_xy(ws, pos: tuple[int,int], value, fmt=None):
    """
    Escribe un valor en pos=(x, y) relativa a table_start_col/row.
    """
    x, y = pos
    ws.write(
        table_start_row + y,
        table_start_col + x,
        value, fmt
    )

In [ ]:
def set_columns_rel(ws, rel_col_widths):
    """
    Aplica ws.set_column a rangos de columnas definidos
    en coordenadas relativas (x) a table_start_col.
    rel_col_widths: lista de tuplas (first_rel, last_rel, width)
    """
    for first_rel, last_rel, width in rel_col_widths:
        abs_first = table_start_col + first_rel
        abs_last  = table_start_col + last_rel
        ws.set_column(abs_first, abs_last, width)

In [ ]:
col_widths_rel = [
    (0, 0, 8),  # columna B en adelante:  B–B ancho=8
    (1, 1, 12),  # C–C ancho=30
    (2, 2, 30),  # D–H ancho=8
    (3, 3, 12),  # I–T ancho=8
    (4, 4, 10),  # U–U ancho=10
    (5, 5, 10),  # U–U ancho=10
    (6, 6, 14),  # U–U ancho=10
    (7, 7, 14),
]

set_columns_rel(ws, col_widths_rel)

### Titulos

In [ ]:
merge_xy(ws, (0, 0), (0, 1), "N.º", wb_formats["header"])